In [1]:
import json
import pandas as pd       
import os
import nltk
from nltk.corpus import stopwords

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
#min_word_count = 40   # Minimum word count                        
#num_workers = 4       # Number of threads to run in parallel
##context = 10          # Context window size   - SkipGram                                                                                 
#context = 5          # Context window size     - CBOW                                                                             
#downsampling = 1e-3   # Downsample setting for frequent words

# Load the model that we created in Part 2 
from gensim.models import Word2Vec
import os


# Open Trining File
os.chdir("/temp/DocumentCorpus/ECHR/echr_dataset-master/echr_dataset-master/document_crawler/IN/")
train = pd.read_csv("TRAIN_classifiedFULL_NOADMISS.tsv", header=0, delimiter="^", quoting=3)
test  = pd.read_csv("TEST_NOclassifiedFULL_NOADMISS.tsv", header=0, delimiter="^", quoting=3)

os.chdir("/temp/DocumentCorpus/ECHR/echr_dataset-master/echr_dataset-master/document_crawler/OUT/AV5/CV/")

train.shape
test.shape

train.columns.values

#model = Word2Vec.load("300features_10minwords_10context")
#model = Word2Vec.load("CBOW_300features_10minwords_10context")
#model = Word2Vec.load("CBOW_300features_10minwords_5context")
model = Word2Vec.load("SG_300features_10minwords_5context")

type(model.wv.syn0)

model.wv.syn0.shape

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5a. Remove extra Stop words
    #moreStopwords = ["b","full","value","file","period","months","additional","basis","street","respect","cases","application","amount","since","costs","address","well","days","series","c","particular","purposes","purpose","proceedings","proceeding","h","able","ba","country","held","board","first","second","final","judge","non","statement","documents","many","notes","note","j","considered","aw","echr","whether","language","ill","time","taken","kh","rovd","must","set","within","p","mr","mrs","provided","sher","one","new","route","routes","three","would","previously","shall","en","k","g","applicants","eur","date","might","paragraph","u","kd","could","made","company","see","public","parking","statements","article","government","business","information","therefore","right","also","applicant","court","act","state","security","section","hearing","v","service","case","law","person","courts","regional","january","february","march","april","may","june","july","august","september","october","november","december"]
    #more_meaningful_words = [token for token in words if token not in moreStopwords]
    #
    # 5. Return a list of words
    #return(more_meaningful_words)
    return(words)


import nltk.data
#nltk.download()   

import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    # raw_sentences = tokenizer.tokenize(review.strip())
    raw_sentences = tokenizer.tokenize(review.strip().decode("utf8"))
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec
    


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
     
    for review in reviews:
       # Print a status message every # review
       if counter%100. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

import time

start = time.time() # Start time

clean_train_reviews = []
for review in train["sentences"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test sentences"
clean_test_reviews = []
for review in test["sentences"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

end = time.time()
elapsed = end - start
print "Time taken for Creating average feature vecs for test sentences: ", elapsed, "seconds."

C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 0 of 4000
Review 100 of 4000
Review 200 of 4000
Review 300 of 4000
Review 400 of 4000
Review 500 of 4000
Review 600 of 4000
Review 700 of 4000
Review 800 of 4000
Review 900 of 4000
Review 1000 of 4000
Review 1100 of 4000
Review 1200 of 4000
Review 1300 of 4000
Review 1400 of 4000
Review 1500 of 4000
Review 1600 of 4000
Review 1700 of 4000
Review 1800 of 4000
Review 1900 of 4000
Review 2000 of 4000
Review 2100 of 4000
Review 2200 of 4000
Review 2300 of 4000
Review 2400 of 4000
Review 2500 of 4000
Review 2600 of 4000
Review 2700 of 4000
Review 2800 of 4000
Review 2900 of 4000
Review 3000 of 4000
Review 3100 of 4000
Review 3200 of 4000
Review 3300 of 4000
Review 3400 of 4000
Review 3500 of 4000
Review 3600 of 4000
Review 3700 of 4000
Review 3800 of 4000
Review 3900 of 4000
Creating average feature vecs for test sentences
Review 0 of 2348
Review 100 of 2348
Review 200 of 2348
Review 300 of 2348
Review 400 of 2348
Review 500 of 2348
Review 600 of 2348
Review 700 of 2348
Review 800 of

In [2]:
#from tempfile import TemporaryFile
#outfile = trainDataVecs
#
## Fit a random forest to the training data, using 100 trees
#from sklearn.ensemble import RandomForestClassifier
#forest = RandomForestClassifier( n_estimators = 100 )
#
#print "Fitting a random forest to labeled training data..."
#forest = forest.fit( trainDataVecs, train["AV3"] )
#
## Test & extract results 
#result = forest.predict( testDataVecs )

from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

grid_values = {} #{'C':[30]} # Decide which settings you want for the grid search. 

model_LR = GridSearchCV(RandomForestClassifier(n_estimators = 100), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 

# Try to set the scoring on what the contest is asking for. 
# The contest says scoring is for area under the ROC curve, so use this.
                            
model_LR.fit(trainDataVecs, train["AV5"] ) # Fit the model.

model_LR.grid_scores_

## Write the test results 
#output = pd.DataFrame( data={"title":test["title"],"conclusion":test["conclusion"], "AV3predict":result} )
##output.to_csv( "Word2Vec_AverageVectors_300features_10minwords_10context.csv", index=False, quoting=3, escapechar='\\', sep='^' )
#output.to_csv( "Word2Vec_CBOW_AverageVectors_300features_10minwords_10context.csv", index=False, quoting=3, escapechar='\\', sep='^' )

LR_result = model_LR.predict_proba(testDataVecs)[:,1] 
#LR_output = pd.DataFrame(data={"id":test["id"], "AV3":LR_result}) # Create our dataframe that will be written.
LR_output = pd.DataFrame( data={"title":test["title"],"conclusion":test["conclusion"], "AV5_Predict":LR_result} )
#LR_output.to_csv('Logistic_Reg_Proj2.csv', index=False, quoting=3) # Get the .csv file we will submit to Kaggle.
LR_output.to_csv( "A5008_SG_AVG_NSX.csv", index=False, quoting=3, escapechar='\\', sep='^'  )






C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
from sklearn.externals import joblib
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

import time

idx=10000000000000
start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th (for 5 times the vocab size, use 1/25th ) of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 25

idx=0
batch_size=45
# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )
#kmeans_clustering = MiniBatchKMeans(n_clusters=num_clusters, random_state=39, init_size=10758)
#kmeans_clustering = MiniBatchKMeans(n_clusters=num_clusters)
#kmeans_clustering = MiniBatchKMeans(init='k-means++', n_clusters=num_clusters, batch_size=batch_size, n_init=10, max_no_improvement=10, verbose=0)
##idx = kmeans_clustering.fit_transform(word_vectors)
#idx = kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."


joblib.dump(kmeans_clustering,  'doc_cluster.pkl')

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))


Time taken for K Means clustering:  167.354000092 seconds.


In [4]:
model.most_similar("arrest")

[(u'lawful', 0.8448737859725952),
 (u'effecting', 0.8442620038986206),
 (u'fleeing', 0.816905677318573),
 (u'vagrants', 0.7921059727668762),
 (u'arrested', 0.7879447937011719),
 (u'insurrection', 0.7840983867645264),
 (u'arresting', 0.7816579341888428),
 (u'quelling', 0.7780519127845764),
 (u'committing', 0.7724398970603943),
 (u'arrestee', 0.7697550654411316)]

In [5]:
model.similarity('russia','ireland' )

0.48704844215384302

In [6]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( np.any(word_centroid_map.values()[i] == cluster) ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'collapse']

Cluster 1
[u'tying', u'stretch', u'footwear', u'clothing', u'undress', u'restrained', u'shackles', u'clothes', u'bottles', u'immersion', u'touching', u'hurting', u'strapped', u'badly', u'asphyxiation', u'wear', u'plastic', u'belts', u'electric']

Cluster 2
[u'teritoriju', u'kazne', u'pravima', u'evanju', u'upravnom', u'vrhovnog', u'tvu', u'pravice', u'srbije', u'croatia', u'crne', u'titi', u'brez', u'pravilnik', u'enju', u'sporovima', u'slovenije', u'postopku', u'avanju', u'em', u'jugoslavije', u'srs', u'ustavnom', u'ustavnog', u'nepotrebnega', u'utvr', u'dopunama', u'odla', u'drugim', u'izmjenama', u'sudovima', u'suda', u'sudu', u'zakona', u'ure', u'vrijeme', u'ovr', u'zatvora', u'federalist', u'stvarnim', u'varstvu', u'akata', u'pravdnem', u'kih', u'ustava', u'ogm', u'gore', u'vlasni', u'sojenja', u'nih', u'osiguranju', u'obiteljski', u'poslovnik', u'upravnim', u'anja']

Cluster 3
[u'neustadt', u'unabh', u'rodk', u'verwaltungssenat', u'ngiger', u'wiener', u'

In [7]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids


In [8]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["sentences"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["sentences"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1
    
    
## Fit a random forest and extract predictions 
#forest = RandomForestClassifier(n_estimators = 100)
#
## Fitting the forest may take a few minutes
#print "Fitting a random forest to labeled training data..."
#forest = forest.fit(train_centroids,train["AV3"])
#result = forest.predict(test_centroids)

#from sklearn.grid_search import GridSearchCV
#from sklearn.ensemble import RandomForestClassifier

grid_values = {} #{'C':[30]} # Decide which settings you want for the grid search. 

model_LR = GridSearchCV(RandomForestClassifier(n_estimators = 100), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 

# Try to set the scoring on what the contest is asking for. 
# The contest says scoring is for area under the ROC curve, so use this.
                            
model_LR.fit(train_centroids, train["AV5"] ) # Fit the model.

model_LR.grid_scores_



## Write the test results 
#output = pd.DataFrame(data={"title":test["title"],"conclusion":test["conclusion"], "AV3":result})
##output.to_csv( "BagOfCentroids500.csv", index=False, quoting=3, escapechar='\\' , sep='^')
#output.to_csv( "CBOW_BagOfCentroids500.csv", index=False, quoting=3, escapechar='\\' , sep='^')
LR_result = model_LR.predict_proba(test_centroids)[:,1] 
#LR_output = pd.DataFrame(data={"id":test["id"], "AV3":LR_result}) # Create our dataframe that will be written.
LR_output = pd.DataFrame( data={"title":test["title"],"conclusion":test["conclusion"], "AV5_Predict":LR_result} )
#LR_output.to_csv('Logistic_Reg_Proj2.csv', index=False, quoting=3) # Get the .csv file we will submit to Kaggle.
LR_output.to_csv( "A5008_SG_CENT_NSX.csv", index=False, quoting=3, escapechar='\\', sep='^'  )
